In [ ]:
"""
Purpose: To check the progress of version 4
pods running

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-05-19 04:02:10,226 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 04:02:10,229 - settings - Setting database.user to celiib
INFO - 2021-05-19 04:02:10,230 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 04:02:10,234 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-05-19 04:02:10,235 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 04:02:10,247 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-05-19 04:02:10,692 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 04:02:10,706 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 04:02:10,707 - settings - Setting database.user to celiib
INFO - 2021-05-19 04:02:10,708 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 04:02:10,709 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 04:02:10,710 - settings - Setting database.user to celiib
INFO - 2021-05-19 04:02:10,711 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 04:02:10,712 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 04:02:11,198 - settings - Setting enable_python_native_blobs to True


In [3]:
import datajoint as dj
version = 88
nucleus_table_ids = du.nucleus_info_table(version = 88) & "segment_id>0"
minnie, schema = du.configure_minnie_vm()


INFO - 2021-05-19 04:02:11,372 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 04:02:11,373 - settings - Setting database.user to celiib
INFO - 2021-05-19 04:02:11,374 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 04:02:11,378 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 04:02:11,913 - settings - Setting enable_python_native_blobs to True


# The list of segments

In [4]:
minnie.NucleiSegmentsRun4()

segment_id segment id for those to be decimated
864691131559524855
864691131658969700
864691131705161142
864691131771477291
864691131800703080
864691131820044899
864691131828706323
864691131834278152
864691131848667157
864691131906133483


# Decimation

In [5]:
curr_table = (minnie.schema.jobs & "table_name='__decimation'")
curr_table

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decimation,00377f5b109485e869a5defb2f6b9c46,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.83,at-compute004,1,427072,2021-05-17 16:00:35
__decimation,0577fe53aed3f61fa48c14cace77c0d5,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.157,at-node30,1,427051,2021-05-17 16:00:17
__decimation,098e0be9fd9ace0ef11c71e33c872874,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.140,at-node13,1,427070,2021-05-17 15:54:37
__decimation,0b32188cc97c146c758bf0868d25e350,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.155,at-node28,1,427093,2021-05-17 16:00:28
__decimation,0b9a0dc68db9c46ed591d4d6fb9f428d,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.66.4.27,at-compute005,1,427063,2021-05-17 16:00:19
__decimation,17c785a4b4690cb9745c874d38ae6638,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.164,at-node37,1,427054,2021-05-17 16:00:19
__decimation,18cd69c4da59d8e5637c21fb14470057,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.133,at-node6,1,427056,2021-05-17 15:50:59
__decimation,1cf6ccb3ad6fd43fafb08cca317df795,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.83,at-compute004,1,427078,2021-05-17 16:00:25
__decimation,1ee98a5dabebf5e40832306e5e0cf5cb,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.83,at-compute004,1,427082,2021-05-17 16:00:21
__decimation,20007fb0b96ab55246a54530e15b4f19,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.159,at-node32,1,427089,2021-05-17 16:00:21


# Soma Extraction

In [ ]:
minnie.BaylorSegmentCentroid()

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__baylor_segment_centroid'")
curr_table

In [ ]:
len(minnie.BaylorSegmentCentroid()) - 102595

In [ ]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000
key_source =  (((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}") & (
                                                                  du.proofreading_segment_id_restriction()
                                                                 ))
(dj.U("segment_id") & key_source) - (dj.U("segment_id") & minnie.BaylorSegmentCentroid())

# 4) Decomposition

In [ ]:
key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")
                  & du.proofreading_segment_id_restriction())
(dj.U("segment_id") & key_source) - (dj.U("segment_id") & minnie.Decomposition())

In [18]:
len(minnie.Decomposition()) - 77723

INFO - 2021-05-19 15:47:29,974 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 15:47:29,982 - settings - Setting database.user to celiib
INFO - 2021-05-19 15:47:29,983 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 15:47:29,983 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 15:47:29,984 - settings - Setting database.user to celiib
INFO - 2021-05-19 15:47:29,984 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 15:47:29,989 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 15:47:30,756 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 15:47:31,253 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 15:47:31,254 - settings - Setting database.user to celiib
INFO - 2021-05-19 15:47:31,262 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 15:47:31,266 - settings - Setting enable_pytho

53

In [19]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
curr_table
#du.restrict_jobs_table_by_error_substring(curr_table,"SystemExit: SIGTERM received").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,00209cbad4844284a34c87b29daa0990,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.83,at-compute004,1,435227,2021-05-19 10:11:26
__decomposition,016e1e5be5a73ed15fc65f5c8c968aa0,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.159,at-node32,1,435577,2021-05-19 09:51:21
__decomposition,0208b9a0f2826d2c26b033e4c3420751,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.142,at-node15,1,435390,2021-05-19 09:38:16
__decomposition,027a3715e3aa47b940b69560ccb1388d,error,=BLOB=,AttributeError: 'list' object has no attribute 'reshape',=BLOB=,celiib@10.66.4.27,at-compute005,1,435185,2021-05-19 09:04:27
__decomposition,02bada258faa726f300eb949b292f5a6,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.161,at-node34,1,435319,2021-05-19 09:23:16
__decomposition,0573816bcc8ef345cd6ad9cd0380724f,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.177,at-node50,1,435212,2021-05-19 10:11:25
__decomposition,05c15c053e510d1782fc019bcc0d1b79,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.142,at-node15,1,435613,2021-05-19 09:50:30
__decomposition,061b32ba60d99ba66d7efe670f50156e,reserved,=BLOB=,,=BLOB=,celiib@10.66.4.27,at-compute005,1,435567,2021-05-19 09:43:49
__decomposition,07c3951dd3214dd9a3d0d30e6fc5dd88,error,=BLOB=,"Exception: neuron 208816 did not fix the manifold edges (meshlab script failed) with output:QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root' Current Plugins Dir is: /meshlab/src/distrib/plugins Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript code; please delete all the spurious xml files Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript code; please delete all the spurious xml files Opening a file with extention off FilterScript Reading filter with name Remove Duplicate Vertices Reading filter with name Smooths normals on a point sets Reading Param with name K : RichInt Reading Param with name useDist : RichBool Reading filter with name Surface Reconstruction: Screened Poisson Reading filter with name Delete Current Mesh Loading Plugins: Total 104 filtering actions Total 1 io plugins Mesh /notebooks/Auto_Proofreading/Preprocessing_Pipeline/v4/Poisson_temp/neuron_208816.off loaded has 8141 vn 16082 fn output mesh /notebooks/Auto_Proofreading/Preprocessing_Pipeline/v4/Poisson_temp/neuron_208816_poisson.off Apply FilterScript: '/notebooks/Auto_Proofreading/Preprocessing_Pipeline/v4/Poisson_temp/poisson_23675.mls' Starting Script of 4 actionsfilter: Remove Duplicate Vertices no additional memory available!!! memory required: 388368 LOG: 2 Removed 0 duplicated vertices Removed 0 duplicated vertices filter: Smooths normals on a point sets no additional memory available!!! memory required: 388368 Removed 0 duplicated vertices filter: Surface Reconstruction: Screened Poisson no additional memory available!!! memory required: 388368 Removed 0 duplicated vertices Problem with filter: Surface Reconstruction: Screened Poisson Failed to apply script file /notebooks/Auto_Proofreading/Preprocessing_Pipeline/v4/Poisson_temp/poisson_23675.mls",=BLOB=,celiib@10.66.4.27,at-compute005,1,435105,2021-05-19 08:25:43
__decomposition,07c59b759f50a6dfd723f1ed1b289da5,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.81,at-compute003,1,435488,2021-05-19 09:25:47


# DecompositionAxon

In [ ]:
"""
NOTE: quit some jobs at 11:20 so could erase those from the jobs 
schema

"""

In [ ]:
"""
Reminder have to eliminate the jobs that were ended prior to running
the new batch

Already did using: 
# curr_table = (minnie.schema.jobs & "table_name='__decomposition_axon'")
# (curr_table & "timestamp<'2021-05-17 14:00:03'").delete()

"""

In [47]:
len(minnie.DecompositionAxon())

78590

In [ ]:
len(minnie.Decomposition()) - 75967

In [ ]:
minnie.Decomposition()

In [25]:
key_source = minnie.Decomposition() & du.proofreading_segment_id_restriction()
(dj.U("segment_id") & key_source) - (dj.U("segment_id") & minnie.DecompositionAxon())

segment_id segment id unique within each Segmentation
864691135576941829
864691135538047346
864691135373445576
864691135341191601
864691135446618982
864691135918326704
864691135274346981
864691135570705133
864691135162270253
864691135845412852


# Proofreading Table

In [82]:
(minnie.AutoProofreadNeurons4() & "axon_length > 1000" & "cell_type_predicted='excitatory'" )

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version key by which to lookup the decomposition process version,axon_version key by which to lookup the finer axon processing method,multiplicity the total number of neurons that came from the parent segment id,cell_type_predicted morphology predicted by classifier,spine_category,n_axons Number of axon candidates identified,n_apicals Number of apicals identified,axon_length length (in um) of the classified axon skeleton,axon_area # area (in um^2) of the classified axon,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,nuclei_distance the distance to the closest nuclei (even if no matching nuclei found),n_nuclei_in_radius the number of nuclei within the search radius of 15000 belonging to that segment,n_nuclei_in_bbox the number of nuclei within the bounding box of that soma,"soma_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,n_boutons,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,axon_angle_maximum the anlge of an identified axon,spine_density_classifier the number of spines divided by skeletal length for branches analyzed in classification,n_branches_processed the number branches used for the spine density analysis,skeletal_length_processed The total skeletal length of the viable branches used for the spine density analysis,n_branches_in_search_radius the number branches existing in the search radius used for spine density,skeletal_length_in_search_radius The total skeletal length of the branches existing in the search radius used for spine density,run_time the amount of time to run (seconds)
864691134884741114,0,0.25,30.00,0,0,0,4,4,1,excitatory,densely_spined,1

In [81]:
curr_table = (minnie.schema.jobs & "table_name='__auto_proofread_neurons4'")
curr_table#.delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__auto_proofread_neurons4,02d77fd487081d52cebfa5b4768beff5,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,1,454305,2021-05-24 10:38:32
__auto_proofread_neurons4,0584875c394c406a885f94656b914259,error,=BLOB=,Exception: No path to 613.,=BLOB=,celiib@10.28.0.150,at-node23,1,454306,2021-05-24 10:38:54
__auto_proofread_neurons4,0e664bf47a9f2b7ee4eeb4c6bb199527,error,=BLOB=,Exception: No path to 994.,=BLOB=,celiib@10.28.0.81,at-compute003,1,454290,2021-05-24 10:40:17
__auto_proofread_neurons4,0e77709d21ae17e7f7d0516c9bb89eca,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.175,at-node48,1,454291,2021-05-24 10:35:19
__auto_proofread_neurons4,0fb7981628c02b5648c0db236270f331,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.164,at-node37,1,454327,2021-05-24 10:32:10
__auto_proofread_neurons4,131a6508c4dea5a7c93e16aec30e2610,error,=BLOB=,Exception: No path to 439.,=BLOB=,celiib@10.28.0.150,at-node23,1,454306,2021-05-24 10:40:32
__auto_proofread_neurons4,1935baac4294ad9aec7e933de51e6fbb,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,1,454274,2021-05-24 10:41:08
__auto_proofread_neurons4,197fa96087b103ffbe3e9032d8ff7047,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.145,at-node18,1,454331,2021-05-24 10:39:20
__auto_proofread_neurons4,1eb342b098451d7bc71017c7ee47f7bb,error,=BLOB=,Exception: No path to 1167.,=BLOB=,celiib@10.28.0.81,at-compute003,1,454275,2021-05-24 10:37:19
__auto_proofread_neurons4,29e51e499706a01ca596201877a07b04,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,1,454311,2021-05-24 10:40:32


# Inspecting Certain Neurons

In [65]:
(minnie.AutoProofreadNeurons4() & "spine_category='densely_spined'") & "axon_length=0"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version key by which to lookup the decomposition process version,axon_version key by which to lookup the finer axon processing method,multiplicity the total number of neurons that came from the parent segment id,cell_type_predicted morphology predicted by classifier,spine_category,n_axons Number of axon candidates identified,n_apicals Number of apicals identified,axon_length length (in um) of the classified axon skeleton,axon_area # area (in um^2) of the classified axon,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,nuclei_distance the distance to the closest nuclei (even if no matching nuclei found),n_nuclei_in_radius the number of nuclei within the search radius of 15000 belonging to that segment,n_nuclei_in_bbox the number of nuclei within the bounding box of that soma,"soma_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,n_boutons,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,axon_angle_maximum the anlge of an identified axon,spine_density_classifier the number of spines divided by skeletal length for branches analyzed in classification,n_branches_processed the number branches used for the spine density analysis,skeletal_length_processed The total skeletal length of the viable branches used for the spine density analysis,n_branches_in_search_radius the number branches existing in the search radius used for spine density,skeletal_length_in_search_radius The total skeletal length of the branches existing in the search radius used for spine density,run_time the amount of time to run (seconds)
864691134884741114,0,0.25,30.00,0,0,0,4,4,1,excitatory,densely_spined,1

In [66]:
((minnie.DecompositionAxon() & "axon_length>1000") & 
(dj.U("segment_id","split_index") & (minnie.AutoProofreadNeurons4() & "spine_category='densely_spined'" & "axon_length=0")).proj())

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,axon_version key by which to lookup the finer axon processing method,decomposition saved neuron object with high fidelity axon,axon_length length (in um) of the classified axon skeleton,run_time the amount of time to run (seconds)
864691135210010560,0,0.25,30.00,0,0,0,5,=BLOB=,3296.796278021018,510.39
864691135293049356,0,0.25,30.00,0,0,0,5,=BLOB=,1453.2752686353103,289.61
864691135385265493,0,0.25,30.00,0,0,0,5,=BLOB=,1691.1845544888658,297.42
864691135385306709,0,0.25,30.00,0,0,0,5,=BLOB=,1622.0490533575262,325.73
864691135385349205,0,0.25,30.00,5,0,0,5,=BLOB=,1125.0301619828829,205.53
864691135403771886,0,0.25,30.00,0,0,0,5,=BLOB=,1108.1114038324233,740.46
864691135463537989,0,0.25,30.00,5,0,0,5,=BLOB=,1283.9609697996507,1148.12
864691135476172200,0,0.25,30.00,5,0,0,5,=BLOB=,1374.225330887211,287.29
864691135490614759,0,0.25,30.00,5,0,1,5,=BLOB=,1248.320815390603,186.92
864691135495637444,0,0.25,30.00,0,0,0,5,=BLOB=,1551.6272446534388,5025.76


In [69]:
minnie.AutoProofreadStats4() & dict(segment_id=864691135495637444)

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version key by which to lookup the decomposition process version,mesh_faces faces indices that were saved off as belonging to proofread neuron (external storage),axon_faces faces indices that were saved off as belonging to proofread neuron's axon (external storage),axon_skeleton the skeleton of the axon of the final proofread neuorn,dendrite_skeleton the skeleton of the dendrite branches of the final proofread neuorn,neuron_skeleton the skeleton of the entire neuron,axon_on_dendrite_merges_error_area the area (in um ^ 2) of the faces canceled out by filter,axon_on_dendrite_merges_error_length the length (in um) of skeleton distance canceled out by filter,low_branch_clusters_error_area the area (in um ^ 2) of the faces canceled out by filter,low_branch_clusters_error_length the length (in um) of skeleton distance canceled out by filter,dendrite_on_axon_merges_error_area the area (in um ^ 2) of the faces canceled out by filter,dendrite_on_axon_merges_error_length the length (in um) of skeleton distance canceled out by filter,double_back_and_width_change_error_area the area (in um ^ 2) of the faces canceled out by filter,double_back_and_width_change_error_length the length (in um) of skeleton distance canceled out by filter,crossovers_error_area the area (in um ^ 2) of the faces canceled out by filter,crossovers_error_length the length (in um) of skeleton distance canceled out by filter,high_degree_coordinates_error_area the area (in um ^ 2) of the faces canceled out by filter,high_degree_coordinates_error_length the length (in um) of skeleton distance canceled out by filter,high_degree_branching_error_area the area (in um ^ 2) of the faces canceled out by filter,high_degree_branching_error_length the length (in um) of skeleton distance canceled out by filter,axon_webbing_t_merges_error_area the area (in um ^ 2) of the faces canceled out by filter,axon_webbing_t_merges_error_length the length (in um) of skeleton distance canceled out by filter,thick_t_merge_error_area the area (in um ^ 2) of the faces canceled out by filter,thick_t_merge_error_length the length (in um) of skeleton distance canceled out by filter,axon_fork_divergence_error_area the area (in um ^ 2) of the faces canceled out by filter,axon_fork_divergence_error_length the length (in um) of skeleton distance canceled out by filter,n_valid_syn_presyn_for_split,n_valid_syn_postsyn_for_split,n_presyn_error_syn_non_axon,n_presyn_error_syn,n_postsyn_error_syn,total_error_synapses,total_presyns,total_postsyns,total_synapses,perc_error_presyn,perc_error_postsyn,overall_percent_error,limb_branch_to_cancel stores the limb information from,red_blue_suggestions
864691135495637444,0,0.25,30.00,0,0,0,4,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,1167.74,1202.45,nan,nan,nan,nan,17.86,4.31,nan,nan,nan,nan,1164.39,1551.63,0.0,0.0,0.0,0.0,0.0,0.0,0,224,25,234,2,236,234,226,460,1.0,0.0088,0.513,=BLOB=,=BLOB=


In [75]:
minnie.AutoProofreadNeurons4() & dict(segment_id=864691135495637444)

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version key by which to lookup the decomposition process version,axon_version key by which to lookup the finer axon processing method,multiplicity the total number of neurons that came from the parent segment id,cell_type_predicted morphology predicted by classifier,spine_category,n_axons Number of axon candidates identified,n_apicals Number of apicals identified,axon_length length (in um) of the classified axon skeleton,axon_area # area (in um^2) of the classified axon,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,nuclei_distance the distance to the closest nuclei (even if no matching nuclei found),n_nuclei_in_radius the number of nuclei within the search radius of 15000 belonging to that segment,n_nuclei_in_bbox the number of nuclei within the bounding box of that soma,"soma_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,n_boutons,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,axon_angle_maximum the anlge of an identified axon,spine_density_classifier the number of spines divided by skeletal length for branches analyzed in classification,n_branches_processed the number branches used for the spine density analysis,skeletal_length_processed The total skeletal length of the viable branches used for the spine density analysis,n_branches_in_search_radius the number branches existing in the search radius used for spine density,skeletal_length_in_search_radius The total skeletal length of the branches existing in the search radius used for spine density,run_time the amount of time to run (seconds)
864691135495637444,0,0.25,30.00,0,0,0,4,4,1,excitatory,densely_spined,1

In [83]:
du.plot_proofread_neuron(864691134884777978,
                        split_index=0,
                        plot_proofreading_synapses=True)

Synapsse Results:
Total Number of Synapses = 529
n_valid_syn_ids_presyn = 119
n_errored_syn_ids_presyn = 179
n_valid_syn_ids_postsyn = 230
n_errored_syn_ids_postsyn = 1


In [84]:
dir(du)

['DecimatedMeshAdapter',
 'DecompositionAdapter',
 'FacesAdapter',
 'MeshAdapter',
 'Path',
 'SkeletonAdapter',
 'SomasAdapter',
 '__builtins__',
 '__cached__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'adapt_mesh_hdf5',
 'adapter_decomp_obj',
 'adapter_faces_obj',
 'adapter_objects',
 'adapter_skeleton_obj',
 'adapter_somas_obj',
 'apply_spines_to_neuron',
 'attributes_need_resetting',
 'config_celii',
 'configure_minnie_vm',
 'configure_nucleus_table',
 'create_suggested_splits_neuroglancer_spreadsheet',
 'current_nucleus_version',
 'current_validation_segment_id_exclude',
 'current_validation_segment_id_restriction',
 'datajoint_filepath_error_to_filepath',
 'decimated_mesh',
 'decomposition',
 'decomposition_by_segment_id',
 'decomposition_folder',
 'decomposition_with_spine_recalculation',
 'delete_segment_from_proofread_tables',
 'dj',
 'download_and_insert_allen_meshes',
 'download_meshes',
 'ed',
 'faces',
 'fetch_neuron_obj_manual',
 'fetch_neuron

In [70]:
decomp_path = (minnie.DecompositionAxon() & dict(segment_id=864691135495637444)).fetch1("decomposition")
neuron_obj = du.filepath_to_neuron_obj(decomp_path)

Decompressing Neuron in minimal output mode...please wait


In [72]:
import neuron_visualizations as nviz
nviz.plot_axon(neuron_obj)


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [74]:
import proofreading_utils as pru
pru.proofread_neuron_class_predetermined(neuron_obj,
    inh_exc_class="excitatory",
    perform_axon_classification=False,
    plot_limb_branch_filter_with_disconnect_effect=True,
    high_fidelity_axon_on_excitatory=False,
    plot_final_filtered_neuron=False,
    plot_new_axon_limb_correspondence=False,
    plot_new_limb_object=False,
    plot_final_revised_axon_branch=False,
    verbose=True,
    verbose_outline=True,
    return_limb_branch_dict_to_cancel=True,
    filter_list=pru.v4_exc_filters())


*****Using v4 Filters!!!



--- Working on filter 0:
function = axon_on_dendrite_merges
function __name__ = filter_away_axon_on_dendrite_merges
function arguments = {'use_pre_existing_axon_labels': True}


----------------------- Running Filter ------------------
using pre-existing labels for axon-error detection
After disconnecte effect, removed_limb_branch_dict = {'L0': [5, 0, 3, 4, 6, 7, 8, 10, 11, 14, 15, 17, 18, 19, 24, 26, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44], 'L1': [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 27], 'L2': [2, 8]}


Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh


total_sk_distance = 1202.452463687308, total_area = 1167.7426786060228
----------------------- FINISHED Running Filter ------------------



 --Filter axon_on_dendrite_merges Results --

--- Working on filter 1:
function = high_degree_branching
function __name__ = filter_away_high_degree_branching
function arguments = {'offset': 1500, 'comparison_distance': 2000, 'worst_case_match_threshold': 65, 'width_diff_max': 75, 'width_diff_perc': 60, 'match_threshold': 45, 'angle_buffer': 15, 'max_degree_to_resolve': 6, 'max_degree_to_resolve_wide': 8, 'match_method': 'best_match', 'kiss_check': True, 'kiss_check_bbox_longest_side_threshold': 450}


----------------------- Running Filter ------------------


 ----- Working on coordinate 0: [783875.94758356 746298.6428372  724360.60274932]--------
kiss_check = True
For test node 3, sibling nodes were: [ 5 14 16]
overlap = [ 5 14 16]
With test node equal to the downstream node
upstream = 15, downstream_nodes = [ 5 14 16  3]
branches_at_coord = [ 3

Angle between 36 and 47 = 40.45 
Angle between 36 and 48 = 57.47 
Angle between 38 and 47 = 22.99 
Angle between 38 and 48 = 96.75 
Angle between 47 and 48 = 83.58 
Final Matches = [[36, 47], [36, 48], [38, 47]], Final Matches Angle = [40.45, 57.47, 22.99]
matched_edges = [[36, 47], [36, 48], [38, 47]]matched_edges_angles = [40.45, 57.47, 22.99]
Step 2: Edges with worst case scenario matching = 65
Remaining Edges = [(36, 47), (36, 48), (38, 47)], Remaining Nodes = [36, 38, 47, 48]
edges_to_remove_by_width = []
Step 2: Edges after widht mismatch
Remaining Edges = [(36, 47), (36, 48), (38, 47)], Remaining Nodes = [36, 38, 47, 48]
--Working on edge [36 47]--
--Working on edge [36 48]--
--Working on edge [38 47]--
Edge [38 47] is matches definite match threshold with: 
Edge Buffer of 17.460000000000004 (angle_buffer = 15)
Edge Angle of 22.99 (match_threshold = 45)
Step 4: Definite Edges
Remaining Edges = [(36, 48), (38, 47)], Remaining Nodes = [36, 38, 47, 48]
upstream_subgraph = [48 36]
P

Angle between 84 and 85 = 26.63 
Angle between 84 and 86 = 126.27 
Angle between 84 and 88 = 75.02 
Angle between 85 and 86 = 34.86 
Angle between 85 and 88 = 98.32 
Angle between 86 and 88 = 115.03 
Final Matches = [[75, 86], [75, 88], [84, 85], [85, 86]], Final Matches Angle = [52.57, 24.19, 26.63, 34.86]
matched_edges = [[75, 86], [75, 88], [84, 85], [85, 86]]matched_edges_angles = [52.57, 24.19, 26.63, 34.86]
Step 2: Edges with worst case scenario matching = 65
Remaining Edges = [(75, 86), (75, 88), (84, 85), (85, 86)], Remaining Nodes = [75, 84, 85, 86, 88]
edges_to_remove_by_width = []
Step 2: Edges after widht mismatch
Remaining Edges = [(75, 86), (75, 88), (84, 85), (85, 86)], Remaining Nodes = [75, 84, 85, 86, 88]
--Working on edge [75 86]--
--Working on edge [75 88]--
Edge [75 88] is matches definite match threshold with: 
Edge Buffer of 28.38 (angle_buffer = 15)
Edge Angle of 24.19 (match_threshold = 45)
--Working on edge [84 85]--
--Working on edge [85 86]--
Step 4: Definit

Angle between 126 and 128 = 76.33 
Angle between 126 and 129 = 137.45 
Angle between 128 and 129 = 115.03 
Final Matches = [], Final Matches Angle = []
matched_edges = []matched_edges_angles = []
Step 2: Edges with worst case scenario matching = 65
Remaining Edges = [], Remaining Nodes = [125, 126, 128, 129]
edges_to_remove_by_width = []
Step 2: Edges after widht mismatch
Remaining Edges = [], Remaining Nodes = [125, 126, 128, 129]
Step 4: Definite Edges
Remaining Edges = [], Remaining Nodes = [125, 126, 128, 129]
upstream_subgraph = [126]
Possible Connections = [], angles = []
Step 5: Removing worst case edges
Remaining Edges = []
Not doing kiss check because upstream_matches = []
short_thick_endnodes_to_remove = [  4  27  29  95 132 140]
winning_downstream = None,error_downstream = [125 128 129] 
coordinate [850259.26366988 762276.22850844 723825.24935581] had error branches [125 128 129]--------


 ----- Working on coordinate 15: [860164.2315175  729254.90553781 711295.92585079]----

total_sk_distance = 1551.627244653439, total_area = 1164.3862372563065
----------------------- FINISHED Running Filter ------------------



 --Filter high_degree_branching Results --

--- Working on filter 2:
function = axon_webbing_t_merges
function __name__ = filter_away_webbing_t_merges
function arguments = {'child_width_maximum': 75, 'parent_width_maximum': 75, 'axon_only': True, 'error_if_web_is_none': True, 'web_size_threshold': 120, 'web_size_type': 'ray_trace_median', 'web_above_threshold': True}


----------------------- Running Filter ------------------
two_downstream_thin_axon_limb_branch = {}
wide_angled_children= {}
Final web t error limb branch dict = {}
limb_branch_dict_to_cancel = {}
limb_branch_dict was empty so returning original neuron
----------------------- FINISHED Running Filter ------------------



 --Filter axon_webbing_t_merges Results --

--- Working on filter 3:
function = thick_t_merge
function __name__ = filter_away_thick_t_merge
function arguments = {}


KeyboardInterrupt: 

# Getting the Synapses Filtered Away

In [ ]:
return_error_table_entries

keys_to_write_without_version_errors

keys_to_write_errors

AutoProofreadSynapseErrors_keys